### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.automation :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.indicators :refer :all]
            [clojure-backtesting.direct :refer :all]
            [clojure.string :as str]
            [clojure.java.io :as io]
            [clojure.pprint :as pprint]
  ) ;; require all libriaries from core
)

nil

### Import dataset

In [2]:
; path to dataset = "/Volumes/T7/CRSP"
; change it to the relative path to your own dataset
;
(load-dataset "/Volumes/T7/CRSP" "main" add-aprc)

### Initialise portfolio （Go back here everytime you want to restart.）

In [3]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

### Rate of Change (ROC) strategy

ROC = (Closing price - Closing n periods ago) / Closing n periods ago * 100

We'll trade according to the following rules:
- **Buy signal**: when ROC crosses the lower threshold (e.g. -30)
- **Sell signal**: when ROC crosses the upper threshold (e.g. +30

In [8]:
(def n 10)
(def lower-threshold -30)
(def upper-threshold 30)
    
(while (< (compare (get-date) "1981-12-31") 0)

  (let [roc (ROC "14593" 10)]
    (if (< roc lower-threshold)
      (order "14593" 1 :print false)) ; buy signal
    (if (> roc upper-threshold)
      (order "14593" -1 :print false))) ; sell signal
  (update-eval-report)
  (next-date))
(end-order)

[]

In [11]:
(print-order-record 10)


|      :date | :tic |  :price | :aprc | :quantity |
|------------+------+---------+-------+-----------|
| 1980-12-30 | AAPL | 35.1875 | 29.79 |        -1 |
| 1980-12-31 | AAPL | 34.1875 | 29.42 |        -1 |
| 1981-01-02 | AAPL |  34.625 | 29.58 |        -1 |
| 1982-01-04 | AAPL | 22.0625 | 24.32 |         3 |


nil

### Check portfolio record

In [12]:
;; view final portfolio
(print-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A |  1015.82 |


nil

In [13]:
;; view portfolio value and return
(print-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage | :margin |
|------------+------------+------------+----------+-------+-----------+---------|
| 1980-12-15 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1980-12-30 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1980-12-31 |   $1000.37 |      0.00% |    0.02% | $0.00 |      0.00 |   0.00% |
| 1981-01-02 |   $1000.04 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1981-01-05 |   $1000.96 |      0.04% |    0.04% | $0.00 |      0.00 |   0.00% |
| 1981-01-06 |   $1002.67 |      0.07% |    0.12% | $0.00 |      0.00 |   0.00% |
| 1981-01-07 |   $1004.28 |      0.07% |    0.19% | $0.00 |      0.00 |   0.00% |
| 1981-01-08 |   $1005.03 |      0.03% |    0.22% | $0.00 |      0.00 |   0.00% |
| 1981-01-09 |   $1003.10 |     -0.08% |    0.13% | $0.00 |      0.00 |   0.00% |
| 1981-01-12 |   $1003.32 |      0.01% |    0.14% | $0.00 |      0.00 |   0.00% |


nil

### Generate evaluation report

In [14]:
(print-eval-report 30)


|      :date | :tot-value |    :vol |  :r-vol | :sharpe | :r-sharpe | :pnl-pt | :max-drawdown |
|------------+------------+---------+---------+---------+-----------+---------+---------------|
| 1980-12-29 |      $1000 | 0.0000% | 0.0000% | 0.0000% |   0.0000% |      $0 |        0.0000 |
| 1980-12-30 |      $1000 | 0.0093% | 0.0093% | 1.7321% |   1.7321% |      $0 |      100.0000 |
| 1980-12-31 |      $1000 | 0.0071% | 0.0071% | 0.2753% |   0.2753% |      $0 |        0.0000 |
| 1981-01-02 |      $1000 | 0.0000% | 0.0000% | 0.0000% |   0.0000% |      $0 |        0.0000 |
| 1981-01-05 |      $1000 | 0.0177% | 0.0177% | 2.3463% |   2.3463% |      $0 |      100.0000 |
| 1981-01-06 |      $1002 | 0.0314% | 0.0314% | 3.6894% |   3.6894% |      $0 |      100.0000 |
| 1981-01-07 |      $1004 | 0.0345% | 0.0345% | 5.3805% |   5.3805% |      $1 |      100.0000 |
| 1981-01-08 |      $1005 | 0.0320% | 0.0320% | 6.8064% |   6.8064% |      $1 |      100.0000 |
| 1981-01-09 |      $1003 | 0.0473% | 0

nil

### Plot variables

In [16]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [17]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

#'clojure-backtesting.demo/data-to-plot

In [18]:
(first data-to-plot)

{:date "1980-12-15", :tot-value 1000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :margin 0.0, :plot "portfolio"}

In [19]:
(plot data-to-plot :plot :date :daily-ret false)